# importing libraries

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import os

# defining paths and parameters

In [2]:
dataset_path = "dataset"

In [3]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

# loading and pre-processing data

In [4]:
imagePaths = list(paths.list_images(dataset_path))
images = []
labels = []

for img_path in imagePaths:
    # extract the class label from the filename
    label = img_path.split(os.path.sep)[-2]

    # load the input image (224x224) and preprocess it
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    # update the data and labels lists, respectively
    images.append(image)
    labels.append(label)

In [5]:
data = np.array(images, dtype="float32")
labels = np.array(labels)

In [6]:
# perform one-hot encoding on the labels
labelbinarizer = LabelBinarizer()
labels = labelbinarizer.fit_transform(labels)
labels = to_categorical(labels)

In [7]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [8]:
# construct the training image generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

# fine-tuning

In [9]:
bModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the the base model
hModel = bModel.output
hModel = AveragePooling2D(pool_size=(5, 5))(hModel)
hModel = Flatten(name="flatten")(hModel)
hModel = Dense(128, activation="relu")(hModel)
hModel = Dropout(0.5)(hModel)
hModel = Dense(2, activation="softmax")(hModel)

model = Model(inputs=bModel.input, outputs=hModel)

for layer in bModel.layers:
    layer.trainable = False

C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0529 11:06:06.754987  3420 deprecation.py:506] From C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


# training and compiling the model

In [11]:
# compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the head
H = model.fit(
    datagen.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

Epoch 1/20


W0529 11:06:53.749636  3420 deprecation.py:323] From C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


34/34 [==============================] - 294s 9s/step - loss: 0.7480 - acc: 0.6264 - val_loss: 0.3708 - val_acc: 0.8442
Epoch 2/20
34/34 [==============================] - 330s 10s/step - loss: 0.4638 - acc: 0.7753 - val_loss: 0.2070 - val_acc: 0.9348
Epoch 3/20
34/34 [==============================] - 355s 10s/step - loss: 0.3175 - acc: 0.8642 - val_loss: 0.1278 - val_acc: 0.9674
Epoch 4/20
34/34 [==============================] - 332s 10s/step - loss: 0.2664 - acc: 0.8717 - val_loss: 0.1136 - val_acc: 0.9601
Epoch 5/20
34/34 [==============================] - 325s 10s/step - loss: 0.2340 - acc: 0.9099 - val_loss: 0.0799 - val_acc: 0.9819
Epoch 6/20
34/34 [==============================] - 302s 9s/step - loss: 0.2431 - acc: 0.9074 - val_loss: 0.0697 - val_acc: 0.9855
Epoch 7/20
34/34 [==============================] - 300s 9s/step - loss: 0.1888 - acc: 0.9232 - val_loss: 0.0558 - val_acc: 0.9928
Epoch 8/20
34/34 [==============================] - 301s 9s/step - loss: 0.1704 - acc: 0.9

# saving the model

In [ ]:
#save the model
print("saving the mask detector model...")
model.save("model_mask.h5")
print('MODEL SAVED')